# Scaling

## MTZ IO

``ess.nmx`` has ``MTZ`` IO helper functions.
They can be used as providers in a workflow of scaling routine.

They are wrapping ``MTZ`` IO functions of ``gemmi``.

In [ ]:
from ess.nmx.mtz_io import read_mtz_file, mtz_to_pandas, MTZFilePath
from ess.nmx.data import get_small_mtz_samples


small_mtz_sample = get_small_mtz_samples()[0]
mtz = read_mtz_file(MTZFilePath(small_mtz_sample))
df = mtz_to_pandas(mtz)
df.head()

## Build Pipeline

Scaling routine includes:
- Reducing individual MTZ dataset
- Merging MTZ dataset 
- Reducing merged MTZ dataset

These operations are done on pandas dataframe as recommended in ``gemmi``.
And multiple MTZ files are expected, so we need to use ``sciline.ParamTable``.
<!--TODO: Update it to use cyclebane instead of ParamTable if needed.-->

In [ ]:
import sciline as sl
from ess.nmx.mtz_io import mtz_io_providers, mtz_io_params
from ess.nmx.mtz_io import MTZFileIndex, SpaceGroupDesc
from ess.nmx.scaling import scaling_providers
from ess.nmx.scaling import WavelengthBinSize, WavelengthScaled

pl = sl.Pipeline(
    providers=mtz_io_providers+scaling_providers,
    params={
        SpaceGroupDesc: "C 1 2 1",  # Replace with the correct space group if needed
        WavelengthBinSize: 50,  # Replace with the correct bin size if needed
        **mtz_io_params
    },
)

file_path_table = sl.ParamTable(
    row_dim=MTZFileIndex, columns={MTZFilePath: get_small_mtz_samples()}
)

pl.set_param_table(file_path_table)
pl

## Build Workflow

In [ ]:
scaling_nmx_workflow = pl.get(WavelengthScaled)
scaling_nmx_workflow.visualize(graph_attr={"rankdir": "LR"})

## Compute Desired Type

In [ ]:
scaled = scaling_nmx_workflow.compute(WavelengthScaled)
scaled